> This notebook contains the evolutionary dataset first analysis.

In [1]:
import pandas as pd
import numpy as np

import re
import csv

from Bio import AlignIO

from Bio.Align import AlignInfo

from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import os
from helpers import*

### 1. Data Loading

> We have both xls and fasta format for the evolutionary dataset. 

In [2]:
# Data Paths
FASTA_msa_seq = 'original_data/msa/4.Fasta_all_7k.txt'
XLS_msa_seq = 'original_data/msa/Dataset_S4.xls'
XLS_msa_seq_csv = 'original_data/msa/Dataset_S4.csv'

# Read the Excel file and write to CSV
df = pd.read_excel(XLS_msa_seq)
df.to_csv(XLS_msa_seq_csv, index=False)
XLS_msa = pd.read_csv(XLS_msa_seq_csv)



In [3]:
# Loading dataframe
file_path = 'data/BLAT_ECOLX_Ranganathan2015-2500/data.csv'
df = pd.read_csv(file_path)

second_column = df.iloc[:, 1]

# Find the minimum and maximum values
min_value = second_column.min()
max_value = second_column.max()
number_of_lines = len(df.index)

print(f"The minimum value in the second column is: {min_value}")
print(f"The maximum value in the second column is: {max_value}")
print(f"The number of lines in the CSV file is: {number_of_lines}")

The minimum value in the second column is: -3.743276064
The maximum value in the second column is: 0.3557906625
The number of lines in the CSV file is: 4807


In [13]:
file_path1 = 'alignments/Q2NB98.a2m'


count = 0

# Open the file and count '>' instances
with open(file_path1, 'r') as file:
    for line in file:
        # Increment count if '>' is found at the beginning of a line
        if line.startswith('>'):
            count += 1

print(f"The number of '>' instances in the file is: {count}")

The number of '>' instances in the file is: 9255


In [5]:
display(XLS_msa)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,User note: GenBank Accession numbers marked wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Database Source,Sequence ID,GenBank Accession,Primary Structure,GXNCRFLQ Motif,Protein Length,Domain Structure,Functional Cluster,Primary Effector,Primary Effector Gene Ontology,Linker Length,Number of Predicted Transmembrane Helices,Transmembrane Helix Topology,Kingdom,Phylum,Class,Order,Family,Genus,Species
3,Interpro,A0A009EVU7,EXA65901.1,MTSFYNGGTSQEIFNLIGQAAALEDIIDAITVWLGDKIPDALVSVM...,GKNCRILQ,850.0,"[N_terminus : (0, 0) ---> GAF_2 : (9, 166) ---...","['EAL', 'GAF', 'GGDEF', 'LOV', 'PAS']",GGDEF,Cyclic Nucleotide Biosythesis,12.0,0.0,NaN,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,baumannii
4,Interpro,A0A009JQP1,EXB31618.1,MNNNCDDHDTYHIFNLIGQHTELHLILSAASTWLESRISNVLVAIM...,GKTCHFLQ,855.0,"[N_terminus : (0, 0) ---> GAF_2 : (26, 166) --...","['EAL', 'GAF', 'GGDEF', 'LOV', 'PAS']",GGDEF,Cyclic Nucleotide Biosythesis,12.0,0.0,NaN,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,baumannii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6780,OneKP,ZYAX_2075737,KU702188,MEPSNPGNTNNSFPSGNVVRKVLDDSPAIISPLPRDSRGSLEVFNP...,GKNCRFLQ,1083.0,"[N_terminus : (0, 0) ---> TLOV : (286, 663) --...","['Pkinase', 'TLOV']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Pinophyta,Pinopsida,Pinales,Taxaceae,Taxus,cuspidata
6781,OneKP,ZZOL_2003873,KU702189,MESTSGSKQDHYFSQRSSSMPKPVRDSLAQLPRDSRGSLEVFNPQG...,GRNCRFLQ,1008.0,"[N_terminus : (0, 0) ---> TLOV : (232, 597) --...","['Pkinase', 'TLOV']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla
6782,OneKP,ZZOL_2007563,KU702190,MALQNGSLLSSDRKCVRSPRMKNHKLIVSQSLIRSYRASIQDELQK...,GRNCRFLQ,394.0,"[N_terminus : (0, 0) ---> TLOV : (49, 354) ---...",['TLOV'],Short_LOV,Short LOV,0.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla
6783,OneKP,ZZOL_2011001,KU702191#,MLVEVSQFTEGSKDKAVRPNGMPESLIKYDARQKDSVVSSVSELVE...,GKNCRFLQ,465.0,"[N_terminus : (0, 0) ---> LOV : (142, 247) ---...","['LOV', 'Pkinase']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla


###  2. Convert MSA .a2m file to adequate .a2m format

> The alignment requirements are the folowing:
The first step of the EVmutation method is to compute a pairwise model of sequences from a family sequence alignment, e.g. the included example file (example/msa_comparison.a2m). This alignment has to be in aligned FASTA/A2M format and must fulfill the following requirements:
* The target sequence may not contain any gaps.
* Columns that should be excluded from model inference (e.g. too many gaps) must be represented by lowercase characters. Gaps in these positions must be represented by a dot (".") rather than a dash ("-").
* The identifier of the target sequence has to be passed to plmc with the -f parameter (see below)

> The initial .a2m format is generated using the hh-suite repo (which should be downloaded): https://github.com/soedinglab/hh-suite, which allows for conversion between different MSA file formats. The command that should be used is the following where the reformat function is uised: reformat.pl file_to_be_converted.a3m converted_file.a2m

In [22]:
initial_a2m_format = 'alignments/LOV_Q2NB98.a2m'
correct_a2m_format = 'alignments/Q2NB98.a2m'
convert_to_correct_a2m(initial_a2m_format, correct_a2m_format)

>Let's apply this function to the fasta file (LOV domains starting with EL222 aligned.fasta).

In [8]:
homology = calculate_sequence_homology('alignments/Q2NB98.a2m')
print(f"Sequence Homology: {homology}%")

Sequence Homology: 0.0%


In [10]:
LOV_msa_a2m_30gaps_removed = 'alignments/Q2NB98_30perc.a2m'
LOV_msa_a2m_dashes = 'alignments/Q2NB98.a2m'
num_removed, perc_removed = remove_high_gap_sequences(LOV_msa_a2m_dashes, LOV_msa_a2m_30gaps_removed, gap_threshold=30)

print(f"Number of sequences removed: {num_removed}")
print(f"Percentage of sequences removed: {perc_removed:.2f}%")

Number of sequences removed: 4363
Percentage of sequences removed: 47.14%


###  3. Domain of interest range analysis

In [ ]:
lov_data = extract_lov_domain_tuples(XLS_msa)
# print(lov_data)